# Model-map illustration of results

## Introduction

In this demo, we illustrate the use of the model-map illustration of result objects. 

## Exercise 1: Data and RDM handling

In [1]:
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import rsatoolbox

### Load model RDMs
Here the models are different layers of Alexnet.
For each layer, different models of how the fMRI voxels sample the neurons are being considered.

The simulated data were generated in Matlab (Kriegeskorte & Diedrichsen 2016). Thus, we load the Matlab files in .mat format.

For each model-RDM, we obtain the RDM itself, a model name, and a measurement model name. The model name specifies the layer used to generate the RDM. The measurement model name specifies the applied distortions.

In [2]:
matlab_data = io.matlab.loadmat('rdms_inferring/modelRDMs_A2020.mat')
matlab_data = matlab_data['modelRDMs']
n_models = len(matlab_data[0])
model_names = [matlab_data[0][i][0][0] for i in range(n_models)]
measurement_model = [matlab_data[0][i][1][0] for i in range(n_models)]
rdms_array = np.array([matlab_data[0][i][3][0] for i in range(n_models)])
model_rdms = rsatoolbox.rdm.RDMs(rdms_array,
                            rdm_descriptors={'brain_computational_model':model_names,
                                             'measurement_model':measurement_model},
                            dissimilarity_measure='Euclidean'
                           )
matlab_data = io.matlab.loadmat('rdms_inferring/noisyModelRDMs_A2020.mat')
repr_names_matlab = matlab_data['reprNames']
fwhms_matlab = matlab_data['FWHMs']
noise_std_matlab = matlab_data['relNoiseStds']
rdms_matlab = matlab_data['noisyModelRDMs']
repr_names = [repr_names_matlab[i][0][0] for i in range(repr_names_matlab.shape[0])]
fwhms = fwhms_matlab.squeeze().astype('float')
noise_std = noise_std_matlab.squeeze().astype('float')
rdms_matrix = rdms_matlab.squeeze().astype('float')

i_rep = 2 #np.random.randint(len(repr_names)) 
i_noise = 1 #np.random.randint(len(noise_std))
i_fwhm = 0 #np.random.randint(len(fwhms))

rdms_data = rsatoolbox.rdm.RDMs(rdms_matrix[:, i_rep, i_fwhm, i_noise, :].transpose())
models = []
for i_model in np.unique(model_names):
    rdm_m = model_rdms.subset('brain_computational_model', i_model).subset('measurement_model','complete')
    m = rsatoolbox.model.ModelFixed(i_model, rdm_m)
    models.append(m)
models_flex = []
for i_model in np.unique(model_names):
    models_flex.append(rsatoolbox.model.ModelSelect(i_model,
        model_rdms.subset('brain_computational_model', i_model)))



In [ ]:
results_1 = rsatoolbox.inference.eval_fixed(models, rdms_data, method='corr')
train_set, test_set, ceil_set = rsatoolbox.inference.sets_k_fold(rdms_data, k_pattern=3, k_rdm=2)
results_2_cv = rsatoolbox.inference.crossval(models_flex, rdms_data, train_set, test_set,
                                             ceil_set=ceil_set, method='corr')
results_3_full = rsatoolbox.inference.bootstrap_crossval(models_flex, rdms_data, k_pattern=4, k_rdm=2, method='corr', N=100)



/Users/heiko/rsatoolbox/rsatoolbox/util/inference_util.py:146: RuntimeWarning: invalid value encountered in multiply
  rdm_mean = np.empty((1, rdm_vector.shape[1])) * np.nan
 25%|█████████████████████████▎                                                                           | 25/100 [00:15<00:50,  1.48it/s]

In [ ]:
rsatoolbox.vis.map_model_comparison(results_1, error_bars='sem')


In [ ]:
# plot results
rsatoolbox.vis.map_model_comparison(
    results_2_cv, rdms_data=rdms_data, test_pair_comparisons=False,
    test_above_0=False, test_below_noise_ceil=False)

In [ ]:
# plot results
rsatoolbox.vis.map_model_comparison(results_3_full, rdms_data=rdms_data, 
                                    error_bars='sem')